In [18]:
# !pip3 install torch
# !pip3 install pandas

In [1]:
import ignite

ModuleNotFoundError: No module named 'ignite'

In [27]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, Dataset
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss

ModuleNotFoundError: No module named 'ignite'

In [20]:
EMBEDDING_VEC_SIZE = 32
DNA_BASES = 4
BAT_SIZE = 128

In [21]:
class SeqData(Dataset):   
    def __init__(self, sequences, labels):
        self.data = torch.from_numpy(sequences)
        self.labels = torch.tensor(labels, dtype=torch.float)
        
    def __len__(self):
        return len(self.data)   
    
    def __getitem__(self, index):
        label = self.labels[index]
        data_val = self.data[index]
        return data_val,label

In [22]:
def get_data_loader(fname):
    df = pd.read_csv(fname, names=["name", "seq", "class"])
    mapping = {'A': 0, 'T': 1, 'C': 2, 'G': 3}
    def mapping_fn(string):
        x = [mapping[x] for x in string]
        return x
    column = df["seq"].apply(lambda x: mapping_fn(x))
    data = np.zeros((len(df), 300), dtype=np.int64)
    for i, d in enumerate(data):
        data[i, :] = d
    dataset = SeqData(data, df['class'].values)
    data_loader = DataLoader(dataset, batch_size=BAT_SIZE, shuffle=True, drop_last=True)
    return data_loader

In [23]:
class Example(nn.Module):
    def __init__(self):
        super(Example, self).__init__()
        self.embed = nn.Embedding(DNA_BASES, EMBEDDING_VEC_SIZE)
        self.fc = nn.Linear(300 * 32, 1)
    
    def forward(self, x):
        x = self.embed(x)
        x = x.view(BAT_SIZE, -1)
        x = self.fc(x)
        return torch.sigmoid(x)

In [24]:
train_loader = get_data_loader("data/fullset_test.csv")

In [25]:
net = Example()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [26]:
for epoch in range(1):  # loop over the dataset multiple times
    epoch_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):

        # zero the parameter gradients
        optimizer.zero_grad()

        labels = labels.view(BAT_SIZE, 1)
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        epoch_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, epoch_loss / (i + 1)))

print('Finished Training')

Finished Training
